In [1]:
using Pkg; Pkg.activate(".")

  Activating project at `~/code/julia/notebooks`


# The Problem

|     Package    |    Fixed Costs     |    Variable Costs     |
|  :----------:  | :--------------:   | :-----------------:   |
|     A          |        16          |          0.25         |
|     B          |        25          |          0.21         |
|     C          |        18          |          0.22         |

- If a package is selected then the corresponding fixed cost is paid, variable cost is paid per minutes used
- 200 minutes will be used in total
- Any package can be used and switched to anytime.
- What is the optimal combination of the use of these packages?
- This problem is taken from Hamdy Taha's Operations Research Book.

In [2]:
using JuMP, HiGHS

In [3]:
BigM = 10000

10000

In [14]:
m = Model(HiGHS.Optimizer)
begin 
    @variable(m, x1 >= 0)
    @variable(m, x2 >= 0)
    @variable(m, x3 >= 0)
    @variable(m, y1, Bin)
    @variable(m, y2, Bin)
    @variable(m, y3, Bin)
    @objective(m, Min, 0.25*x1 + 0.21*x2 + 0.22*x3 + 16*y1 + 25*y2 + 18*y3)
    @constraint(m, x1 + x2 + x3 == 200)
    @constraint(m, x1 <= BigM * y1)
    @constraint(m, x2 <= BigM * y2)
    @constraint(m, x3 <= BigM * y3)
end 
optimize!(m);

Running HiGHS 1.12.0 (git hash: 755a8e027a): Copyright (c) 2025 HiGHS under MIT licence terms
MIP has 4 rows; 6 cols; 9 nonzeros; 3 integer variables (3 binary)
Coefficient ranges:
  Matrix  [1e+00, 1e+04]
  Cost    [2e-01, 2e+01]
  Bound   [1e+00, 1e+00]
  RHS     [2e+02, 2e+02]
Presolving model
4 rows, 6 cols, 9 nonzeros  0s
4 rows, 6 cols, 9 nonzeros  0s
Presolve reductions: rows 4(-0); columns 6(-0); nonzeros 9(-0) - Not reduced

Solving MIP model with:
   4 rows
   6 cols (3 binary, 0 integer, 0 implied int., 3 continuous, 0 domain fixed)
   9 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic;
     I => Shifting; J => Feasibility jump; L => Sub-MIP; P => Empty MIP; R => Randomized rounding;
     S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution; Y => HiGHS solution;
     Z => ZI Round; l => Trivial lower; p => Trivial point; u => Trivial upper; z => Trivial zero

        Nodes      |    B&B Tree     |            Objecti

In [15]:
println(JuMP.LazyString(m))

Min 0.25 x1 + 0.21 x2 + 0.22 x3 + 16 y1 + 25 y2 + 18 y3
Subject to
 x1 + x2 + x3 = 200
 x1 - 10000 y1 ≤ 0
 x2 - 10000 y2 ≤ 0
 x3 - 10000 y3 ≤ 0
 x1 ≥ 0
 x2 ≥ 0
 x3 ≥ 0
 y1 binary
 y2 binary
 y3 binary



In [16]:
value.([x1, x2, x3])

3-element Vector{Float64}:
   0.0
   0.0
 200.0

In [17]:
value.([y1, y2, y3])

3-element Vector{Float64}:
 -0.0
 -0.0
  1.0

In [18]:
JuMP.objective_value(m)

62.0